In [48]:
import pandas as pd
import requests

In [35]:
df = pd.read_csv('funders.csv')

In [41]:
country_mapping= {
        'United States': 'US', 'Canada': 'CA', 'Netherlands': 'NL',
        'Australia': 'AU', 'Japan': 'JP', 'France': 'FR',
        'Sweden': 'SE', 'Switzerland': 'CH', 'United Kingdom': 'GB',
        'Germany': 'DE', 'Brazil': 'BR', 'European Union': 'EU',
        'Finland': 'FI', 'Denmark': 'DK', 'Portugal': 'PT',
        'Spain': 'ES', 'Italy': 'IT', 'Poland': 'PL',
        'Austria': 'AT', 'Czech Republic': 'CZ', 'China': 'CN',
        'South Korea': 'KR', 'Taiwan': 'TW', 'India': 'IN',
        'Argentina': 'AR', 'Chile': 'CL', 'Israel': 'IL',
        'Turkey': 'TR', 'Colombia': 'CO', 'Mexico': 'MX',
        'Peru': 'PE', 'Kenya': 'KE', 'Nigeria': 'NG',
        'Malaysia': 'MY', 'Thailand': 'TH', 'Vietnam': 'VN',
        'International': 'INT', 'Qatar': 'QA', 'Uruguay': 'UY',
        'South Africa': 'ZA', 'Ecuador': 'EC', 'Costa Rica': 'CR',
        'Uganda': 'UG', 'Tanzania': 'TZ', 'International (Africa)': 'AFR',
        'Indonesia': 'ID', 'Philippines': 'PH', 'Pakistan': 'PK',
        'Bangladesh': 'BD', 'Sri Lanka': 'LK', 'Nepal': 'NP',
    'Russia': 'RU'
    }

def search_openalex(query, country_code=None):
    """
    Recherche un funder dans l'API OpenAlex
    
    Args:
        query: Terme de recherche (nom ou acronyme)
        country_code: Code pays pour filtrer (optionnel)
    
    Returns:
        dict ou None: Informations du funder trouvé
    """
    url = "https://api.openalex.org/funders"
    params = {
        'search': query,
        'per_page': 5,
        'mailto': 'your-email@example.com'  # Recommandé par OpenAlex
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data.get('results'):
            # Si on a un code pays, chercher correspondance exacte
            if country_code:
                for result in data['results']:
                    result_country = result.get('country_code', '').upper()
                    if result_country == country_code.upper():
                        return result
            # Sinon, prendre le premier résultat
            return data['results'][0]
        return None

    except requests.exceptions.RequestException as e:
        print(f"  ⚠️  Erreur réseau: {e}")
        return None
    except Exception as e:
        print(f"  ⚠️  Erreur: {e}")
        return None


In [39]:
data = df.to_dict(orient='records')
len(data)

117

In [53]:
for d in data:
    country_code = country_mapping.get(d['country'], '')
    search_query = f"{d['name']} {d['acronym']}"
    result = search_openalex(search_query, country_code)
    if result:
        for f in ['id', 'display_name', 'alternate_titles', 'description', 'homepage_url', 'image_thumbnail_url' ]:
            d[f'openalex_{f}'] = result[f]
        if 'ids' in result:
            for t in ['ror', 'wikidata']:
                if t in results['ids']:
                    d[t] = results['ids'][t]

  ⚠️  Erreur: 'NoneType' object has no attribute 'upper'
  ⚠️  Erreur: 'NoneType' object has no attribute 'upper'


In [57]:
pd.DataFrame(data).to_csv('funders_with_openalex_id.csv', index=False)

In [58]:
len(data)

117